**40. Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o igual a 1, para aquellos que editaron una de las versiones actuales del contenido, ¿Cuál es la tasa de revisiones sin comentario que realizan en función de los idiomas que saben? (⭐⭐⭐)**

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=63b393cda016e32380a1996cf8b8aeb28a79d6ac7a04f374292f83b1ec128b7e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:7 https://developer.download.nvidia.

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)

df_languages = sqlContext.read.csv('/content/drive/MyDrive/datosTP1/languages.csv', header=True, inferSchema=True)
rdd_languages = df_languages.rdd

df_contents = sqlContext.read.csv('/content/drive/MyDrive/datosTP1/contents.csv', header=True, inferSchema=True)
rdd_contents = df_contents.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
rdd_languages.take(5)

[Row(babel_user=4502458, babel_lang='FR', babel_level='2'),
 Row(babel_user=5928200, babel_lang='aa', babel_level='0'),
 Row(babel_user=46918, babel_lang='ab', babel_level='0'),
 Row(babel_user=2050449, babel_lang='ab', babel_level='1'),
 Row(babel_user=4715583, babel_lang='ace', babel_level='0')]

In [7]:
rdd_contents.take(5)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id='10', namespace='0', revision_id='137966826', parent_revision='137825165', revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id='4642283', revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id='15', namespace='0', revision_id='137986788', parent_revision='133666383', revision_timestamp='2021-08-29T17:16:00Z',

Me quedo con los revisores con ids válidos y les asigno True si hicieron comentario y False en caso contrario.

In [8]:
def hizo_comentario(comentario):
  if(comentario == None or comentario == ""):
    return False

  return True 

revisores_con_comentario = rdd_contents.filter( lambda x: x.revisor_id != None and x.revisor_id.isdigit() ).map( lambda x: (int(x.revisor_id), hizo_comentario(x.revisor_comment)) )

In [9]:
revisores_con_cantidad_idiomas = rdd_languages.filter(lambda x: x[2] != "0").map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x + y)

Hago un join para quedarme con los revisores, cuantos idiomas hablan y si hicieron comentario o no. Luego me quedo con una tupla con la cantidad de idiomas hablados como clave y si hizo comentario o no como valor. Lo cacheo porque lo voy a reutilizar.

In [10]:
cantidad_idiomas_con_comentarios = revisores_con_comentario.join(revisores_con_cantidad_idiomas).map(lambda x: (x[1][1], x[1][0]) ).cache()

In [11]:
cantidad_idiomas_con_comentarios.take(3)

[(4, True), (4, True), (4, True)]

Para los que hablan la misma cantidad de idiomas, sumo las revisiones que no tienen comentario.

In [12]:
cantidad_sin_comentarios_por_cantidad_idiomas = cantidad_idiomas_con_comentarios.filter(lambda x: x[1] != True).map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)

Para los que hablan la misma cantidad de idiomas, sumo todas las revisiones hechas.

In [13]:
cantidad_comentarios_por_cantidad_idiomas = cantidad_idiomas_con_comentarios.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)

In [14]:
cantidad_comentarios_por_cantidad_idiomas.join(cantidad_sin_comentarios_por_cantidad_idiomas).map(lambda x: (x[0], (x[1][1] / x[1][0]) * 100) ).takeOrdered(20, key=lambda x: x[0])

[(1, 24.89639885637187),
 (2, 19.30673968510797),
 (3, 12.955704377300453),
 (4, 14.553954031943903),
 (5, 13.079006173540886),
 (6, 16.710434642799722),
 (7, 6.788203086112493),
 (8, 26.430627133391337),
 (9, 22.47191011235955),
 (10, 9.652572482438773),
 (11, 3.7037037037037033),
 (12, 4.801720019111324),
 (13, 33.33333333333333),
 (14, 22.15821812596006),
 (16, 17.582417582417584),
 (17, 14.166666666666666),
 (21, 50.0),
 (22, 69.74789915966386),
 (24, 5.0936768149882905),
 (34, 14.965986394557824)]

In [ ]:
##############################################

**Abajo es lo mismo, pero en vez de tasa por CANTIDAD DE IDIOMAS HABLADOS, hago tasa por IDIOMA PUNTUAL** (lo había hecho así al principio).

In [ ]:
##############################################

In [15]:
lenguajes = rdd_languages.filter(lambda x: x[2] != "0").map(lambda x: (x[0], x[1].lower()))

Uno los revisores con los lenguajes que hablan, usando el id que ambos comparten.

In [16]:
revisores_con_lenguaje = lenguajes.join(revisores_con_comentario)

Creo un rdd con tuplas que tengan al lenguaje como clave y su cantidad de revisiones total como valor. Lo cacheo porque tarda un poco el cálculo, así no hacerlo devuelta.

In [17]:
revisiones_por_lenguaje = revisores_con_lenguaje.map(lambda x: (x[1][0], 1) ).reduceByKey(lambda x,y: x+y).cache()

In [18]:
revisiones_por_lenguaje.take(3)

[('da', 217), ('no', 4522), ('gl', 65213)]

Creo un rdd con tuplas que tengan al lenguaje como clave y su cantidad de revisiones sin comentario total como valor.

In [19]:
sin_comentario_por_lenguaje = revisores_con_lenguaje.filter(lambda x: x[1][1] != True).map(lambda x: (x[1][0], 1)).reduceByKey(lambda x,y: x+y)

In [20]:
sin_comentario_por_lenguaje.take(3)

[('gl', 9847), ('grc', 475), ('no', 81)]

Hago un join con la clave siendo el lenguaje y divido (cantidad de revisiones sin comentario / cantidad total de revisiones) * 100, para cada idioma.

In [21]:
revisiones_por_lenguaje.join(sin_comentario_por_lenguaje).map(lambda x: (x[0], (x[1][1] / x[1][0]) * 100) ).collect()

[('da', 34.56221198156682),
 ('no', 1.7912428129146396),
 ('gl', 15.099750049836688),
 ('grc', 4.704367633950678),
 ('sq', 40.0),
 ('lt', 13.114754098360656),
 ('cs', 9.704790525013259),
 ('hr', 6.1935140882509305),
 ('sl', 5.979323833473037),
 ('sgs', 17.97752808988764),
 ('pms', 27.27272727272727),
 ('uz', 14.89841986455982),
 ('co', 25.0),
 ('lzh', 100.0),
 ('zh-hant', 100.0),
 ('tl', 18.014705882352942),
 ('sco', 13.354632587859424),
 ('sc', 19.565217391304348),
 ('csb', 50.0),
 ('eml', 50.0),
 ('az', 14.590747330960854),
 ('vsv', 84.90566037735849),
 ('kw', 33.33333333333333),
 ('su', 22.22222222222222),
 ('ka', 7.02576112412178),
 ('mnw', 100.0),
 ('kn', 0.5208333333333333),
 ('qwh', 37.77777777777778),
 ('ast', 15.114054451802797),
 ('be', 7.975630019385211),
 ('la', 14.868586401433118),
 ('oc', 14.833005893909625),
 ('pt', 11.262958614349817),
 ('ro', 13.338246131171703),
 ('ang', 0.13966480446927373),
 ('zh', 9.005867845905264),
 ('bg', 20.163636363636364),
 ('fi', 8.050089445